# Mistake analysis
This notebook gives a quick look into the predications the model makes that are wrong according to the reference evaluation data. 

In [20]:
from transformers import (
    T5Tokenizer,
    AutoTokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
import torch
from datasets import load_dataset, concatenate_datasets
from wasabi import msg
import yaml
import re
import pandas as pd
from pathlib import Path
from os.path import abspath
from tqdm import tqdm
from typing import Set, Dict, Tuple, Union
from helper_functions import *

## Setting home directory

In [2]:
home_dir = Path(abspath("")).parent
config_path = home_dir.joinpath("config/config_T5-3b_cdr.yaml")
trained_model_path = home_dir.parent.joinpath("data/generative_re_model_storage_azure/47/checkpoints/checkpoint-100")
predictions_dataset_dir = home_dir.joinpath("data/evaluate_trained_model")

msg.info(f"Home directory: {home_dir}")
msg.info(f"Selected config: {config_path}")
msg.info(f"Selected model: {trained_model_path}")

ℹ Home directory: /home/lgrootde/Generative-re-tests
ℹ Selected config:
/home/lgrootde/Generative-re-tests/config/config_T5-3b_cdr.yaml
ℹ Selected model:
/home/lgrootde/data/generative_re_model_storage_azure/47/checkpoints/checkpoint-100


## Load Config & Dataset

In [3]:
# Load the config
with open(config_path) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [4]:
dataset = load_dataset(
        config['dataset_vars']['type'], 
        data_dir=home_dir.joinpath(config['dataset_vars']['dir']),
        column_names=config['dataset_vars']['column_names']
        )

dataset_eval = concatenate_datasets([dataset[split].select(range(1,501)) for split in config["splits_for_validation"]]) 

## Load trained model and tokenizer

In [5]:
use_model = False

In [6]:
if use_model:
    model_name = config['model_name']
    device_map = {"": 0}
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, legacy=False)
    
    # Load model after training
    model = T5ForConditionalGeneration.from_pretrained(
        trained_model_path,
        device_map=device_map,
        local_files_only=True
    )

## Run model over evaluation dataset 
And note where the model makes mistakes and where the model prediction match the reference.

In [7]:
if use_model:
    # run model over dataset
    data = {
        "input":[],
        "expected output":[],
        "predicted output":[],
        "match":[]
    }
    
    for row in tqdm(dataset_eval):
        # Generate prediction
        input_ids = tokenizer(row['input'], return_tensors="pt").input_ids.to('cuda') 
        output = model.generate(input_ids, max_new_tokens=config["generation_max_length"])
        predicted = tokenizer.decode(output[0], skip_special_tokens=True)
    
        # check for match
        if row["relations"].strip() == predicted.strip():
            match = True
        else:
            match = False
    
        # Save all information
        data["input"].append(row["input"])
        data["expected output"].append(row["relations"])
        data["predicted output"].append(predicted)
        data["match"].append(match)
        
    dataframe = pd.DataFrame.from_dict(data)

In [12]:
# Save dataframe   
dataset_name = ("run"+
                trained_model_path.parts[-3]+
                "_"+
                trained_model_path.parts[-1]+
                "_"+
                "on"+
                "_"+
                Path(config['dataset_vars']['dir']).parts[-1]+
                "_"+
                "_".join(config["splits_for_validation"])+
                ".csv")
    
msg.info(f"dataset name: {dataset_name}")

ℹ dataset name: run47_checkpoint-100_on_cdr_seq2rel_validation.csv


In [8]:
if use_model:
    # Save dataframe
    dataframe.to_csv(predictions_dataset_dir.joinpath(dataset_name))

## Show data for manual inspection

In [13]:
# Load dataset:
dataset_path = predictions_dataset_dir.joinpath(dataset_name)
dataframe = pd.read_csv(dataset_path)

In [27]:
def highlight_entities(input_text: str, rels_expected: Set[Dict], rels_predicted: Set[Dict]) -> str:
    # Create a set to store all entity texts for predicted relationships
    entity_texts_predicted = set()
    
    # Extract all entity texts from the predicted relationships data
    for rel in rels_predicted:
        head_ent_text = rel['head_ent']['text']
        if isinstance(head_ent_text, tuple):
            for text in head_ent_text:
                entity_texts_predicted.add(text)
        else:
            entity_texts_predicted.add(head_ent_text)
        
        tail_ent_text = rel['tail_ent']['text']
        if isinstance(tail_ent_text, tuple):
            for text in tail_ent_text:
                entity_texts_predicted.add(text)
        else:
            entity_texts_predicted.add(tail_ent_text)

    # Highlight the entity texts for predicted relationships in the input text with pastel blue color
    highlighted_text = input_text
    for entity_text in entity_texts_predicted:
        highlighted_text = re.sub(r'\b{}\b'.format(re.escape(entity_text)),
                                  '\033[48;2;173;216;230;38;2;0;0;0m{}\033[00m'.format(entity_text),
                                  highlighted_text, flags=re.IGNORECASE)
    
    # Create a set to store all entity texts for expected relationships
    entity_texts_expected = set()
    
    # Extract all entity texts from the expected relationships data
    for rel in rels_expected:
        head_ent_text = rel['head_ent']['text']
        if isinstance(head_ent_text, tuple):
            for text in head_ent_text:
                entity_texts_expected.add(text)
        else:
            entity_texts_expected.add(head_ent_text)
        
        tail_ent_text = rel['tail_ent']['text']
        if isinstance(tail_ent_text, tuple):
            for text in tail_ent_text:
                entity_texts_expected.add(text)
        else:
            entity_texts_expected.add(tail_ent_text)

    

    # Highlight the entity texts for expected relationships in the input text with pastel green color
    for entity_text in entity_texts_expected:
        highlighted_text = re.sub(r'\b{}\b'.format(re.escape(entity_text)),
                                  '\033[48;2;144;238;144;38;2;0;0;0m{}\033[00m'.format(entity_text),
                                  highlighted_text, flags=re.IGNORECASE)

    return highlighted_text

In [28]:
print("Entities of expected output colored \033[48;2;144;238;144;38;2;0;0;0mgreen\033[00m, of predicted \033[48;2;173;216;230;38;2;0;0;0mblue\033[00m\n")
for row in dataframe[dataframe["match"]==False].iterrows():
    msg.info("input text")
    rels_expected=extract_relation_triples(row[1]["expected output"], config["ner_labels"], config["re_labels"], True)
    rels_predicted=extract_relation_triples(row[1]["predicted output"], config["ner_labels"], config["re_labels"], True)
    print(highlight_entities(row[1]["input"], rels_expected, rels_predicted))
    msg.good("Expected output:")
    print(row[1]["expected output"])
    msg.info("Actual output:")
    print(row[1]["predicted output"])
    print("\n\n")

Entities of expected output colored green, of predicted blue

ℹ input text
tricuspid valve regurgitation and lithium carbonate toxicity in a newborn infant. A newborn with massive tricuspid regurgitation, atrial flutter, congestive heart failure, and a high serum lithium level is described. This is the first patient to initially manifest tricuspid regurgitation and atrial flutter, and the 11th described patient with cardiac disease among infants exposed to lithium compounds in the first trimester of pregnancy. Sixty-three percent of these infants had tricuspid valve involvement. lithium carbonate may be a factor in the increasing incidence of congenital heart disease when taken during early pregnancy. It also causes neurologic depression, cyanosis, and cardiac arrhythmia when consumed prior to delivery.
✔ Expected output:
lithium carbonate @CHEMICAL@ neurologic depression @DISEASE@ @CID@ lithium carbonate @CHEMICAL@ cyanosis @DISEASE@ @CID@ lithium carbonate @CHEMICAL@ cardiac arrhythm

ValueError: Text is unstructured: 'oestrogens @CHEMICAL@ venous thrombo-embolism @DISEASE@ @CID@ progestogens @CHEMICAL@ venous thrombo-embolism @DISEASE@ @CID@ progestogens @CHEMICAL@ dementia @DISEASE@ @CID@ progestogens @CHEMICAL@ venous thrombo-embolism @DISEASE@ @CID@ progestogens @CHEMICAL@ stroke @DISEASE@ @CID@ progestogens @CHEMICAL@ breast cancer @DISEASE@ @CID@ progestogens @CHEMICAL@ stroke @DISEASE@ @CID@ progestogens @CHEMICAL@ breast cancer @DISEASE@ @CID@ progestogens @CHEMICAL@ stroke @DISEASE@ @CID@ progestogens @CHEMICAL@ stroke @DISEASE@ @CID@ progestogens @CHEMICAL@ stroke @DISEASE@ @CID@ progestogens @'
Text should end with a relationship label found in re_labels: ['@CID@'].
